In [1]:
from pathlib import Path

import util
from util import p_, workflow

import automech

file = util.notebook_file() if util.is_notebook() else __file__
tag = util.file_tag(file)
root_path = Path("..")
par_mech = workflow.read_parent_mechanism(root_path=root_path)

In [2]:
# # Generate submechanism
# from automol.graph import enum

# gen_mech1 = workflow.previous_version_species(tag=tag, root_path=root_path)
# #  - Migrations
# gen_mech1 = automech.enumerate_reactions(
#     gen_mech1, enum.ReactionSmarts.qooh_formation, src_mech=par_mech
# )
# #   - Ring-forming scissions
# gen_mech1 = automech.enumerate_reactions(
#     gen_mech1,
#     enum.ReactionSmarts.qooh_ring_forming_scission,
#     src_mech=par_mech,
#     excl_rcts=automech.unstable_species_names(gen_mech1),
# )
# #  - Ring-opening scissions
# gen_mech1 = automech.enumerate_reactions(
#     gen_mech1,
#     enum.ReactionSmarts.ring_beta_scission,
#     src_mech=par_mech,
#     rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
#     excl_rcts=automech.unstable_species_names(gen_mech1),
# )
# #  - Beta scissions
# gen_mech1 = automech.enumerate_reactions(
#     gen_mech1,
#     enum.ReactionSmarts.beta_scission,
#     src_mech=par_mech,
#     rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
#     excl_rcts=automech.unstable_species_names(gen_mech1),
# )
# #  - Beta scissions
# gen_mech1 = automech.enumerate_reactions(
#     gen_mech1,
#     enum.ReactionSmarts.beta_scission,
#     src_mech=par_mech,
#     rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
#     excl_rcts=automech.unstable_species_names(gen_mech1),
# )
# #  - Vinyl beta scissions
# gen_mech1 = automech.enumerate_reactions(
#     gen_mech1,
#     enum.ReactionSmarts.vinyl_beta_scission,
#     src_mech=par_mech,
#     rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
#     excl_rcts=automech.unstable_species_names(gen_mech1),
# )
# #  - Instabilities
# gen_mech1 = automech.enumerate_products(
#     gen_mech1,
#     enum.ReactionSmarts.qooh_instability,
#     src_mech=par_mech,
#     rcts_=[automech.species.names(gen_mech1.species, fml="C*H*O2")],
# )
# automech.display(gen_mech1, height="1000px")

In [3]:
# # Expand stereochemistry
# ste_mech1, *_, gen_mech1 = workflow.expand_stereo(
#     mech=gen_mech1, tag=tag, root_path=root_path
# )

In [4]:
# # Update previous version
# gen_mech, ste_mech = workflow.update_previous_version(
#     gen_mech=gen_mech1, ste_mech=ste_mech1, tag=tag, root_path=root_path
# )
# automech.display(gen_mech, height="1000px")

In [5]:
# # Prepare calculation
# workflow.prepare_calculation(
#     gen_mech=gen_mech, ste_mech=ste_mech, tag=tag, root_path=root_path
# )

In [6]:
# Gather statistics
workflow.gather_statistics(tag=tag, root_path=root_path)

New species and reactions:

SPECIES

C5O2qarhfr  ! SMILES: C=CCC[CH]OO  AMChI: AMChI=1/C5H9O2/c1-2-4-5-3-7-6/h2-3,6H,1,4-5H2

END


REACTIONS    CAL/MOL   MOL

S(731) = C2H4(52) + S(2258)              1.000      0.000      0.000
S(728) = C2H4(52) + S(2258)              1.000      0.000      0.000
S(725) = C5O2qarhfr                      1.000      0.000      0.000

END



Statistics summary:

shape: (5, 3)
┌────────────────────┬───────────────┬────────────────┐
│ Mechanism          ┆ Species Count ┆ Reaction Count │
│ ---                ┆ ---           ┆ ---            │
│ str                ┆ i64           ┆ i64            │
╞════════════════════╪═══════════════╪════════════════╡
│ Parent             ┆ 2534          ┆ 10002          │
│ Generated          ┆ 21            ┆ 16             │
│ Stereo             ┆ 27            ┆ 34             │
│ Generated - Parent ┆ 1             ┆ 3              │
│ Stereo - Parent    ┆ 1             ┆ 8              │
└────────────────────┴────────

In [7]:
# Prepare simulation
workflow.prepare_simulation(tag=tag, root_path=root_path)
workflow.prepare_simulation_species(tag=tag, root_path=root_path)


Reading mechanisms...

Adding calculated thermo...

Adding calculated rates...


InvalidOperationError: `old` input for `replace` must not contain duplicates

In [ ]:
# Check rates
workflow.plot_rates(tag=tag, root_path=root_path)

In [ ]:
# # Plot simulation
# chart_dct = workflow.plot_simulation(
#     tag=tag,
#     x_col="O2_molecules",
#     root_path=root_path,
#     line_source_=["Hill", "Lokachari"],
#     point_source="Experiment",
#     control=False,
# )
# mech = automech.io.read(
#     p_.full_calculated_mechanism(tag, "json", path=p_.data(root_path))
# )
# for name, chart in chart_dct.items():
#     automech.display_species(mech, [name])
#     chart.show()